In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis


In [2]:
model_type = "ffn"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
outdir_suffix = "dev2"
args.n_epochs = 4000 
args.init_model = "{}-base-nopen-gumbel1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 50 

args.penalty_gumbel = True 

args.scale_lambdas = True
args.connectivity = False 
args.valency = False 
args.euler_characteristic_penalty = False

args.conn_lambda_start = 0.001
args.conn_lambda_end = 25
args.conn_lambda_epochs_start = 4001
args.conn_lambda_epochs_end = 8000
args.valency_lambda_start = 0.001
args.valency_lambda_end = 100
args.valency_lambda_epochs_start = 8000
args.valency_lambda_epochs_end = 14000
args.euler_characteristic_penalty = False
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 10000


args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=50, conn_lambda_end=25, conn_lambda_epochs_end=8000, conn_lambda_epochs_start=4001, conn_lambda_start=0.001, conn_penalty_function='capped_logdet', connectivity=False, connectivity_hard=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=False, euler_lambda_end=100, euler_lambda_epochs_end=50, euler_lambda_epochs_start=1, euler_lambda_start=1, ffn_activation='LeakyReLU', init_decoder_model='ffn-base-nopen-gumbel1_decode', init_model='ffn-base-nopen-gumbel1', linear_out=False, model_type='ffn', n_epochs=4000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=300, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_dev2/ffn-base-nopen-gumbel1', pc_hidden=100, penalty_gumbel=True, pred_hidden=150, scale_lambdas=True, sinkhorn_entropy=0.1, sinkhorn_max_it=10000, task

In [3]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
train_data_loader = get_loader("molgen/data/chembl50", "train", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

In [6]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

Found previous model mol_opt/output_dev2/ffn-base-nopen-gumbel1/model_ffn-base-nopen-gumbel1_1113, epoch 1113. Overwriting args.
Epoch: 1114


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


FGW torch.Size([29508, 5]) 7.678664405830204e-05
Penalty params: tau=0.59866 conn_l=0.00100 val_l=0.00100 euler_l=1.00000 epoch=1114 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 1114, train
 fgw:0.2151746
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2151746
Measure Epoch 1114, train
 similarity:0.0274192
 penlog:-92.2122034
Metrics Epoch 1114, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:98.8695652
 batch_molecular_disconnected_validity:8.0000000
 batch_connected_components:2.7200000
 batch_invalid_valency_nodes:30.4347826
 batch_nodes_0degree:1.2200000
 batch_nodes_7plus_degree:1.6200000
 invalid_euler_toofew:8.0000000
 invalid_euler_toomany:30.0000000
 avg_euler_error:4.0400000
 batch_node_degree:3.1095652
Logits [20.261760711669922, 2.3889834880828857, 68.16580963134766]
Epoch duration: 3.1865692138671875
mol_opt/output_dev2/ffn-base-nopen-gumbel1/model_ffn-base-nopen-gumbel1_1114
Epoc

/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


FGW torch.Size([29508, 5]) 6.966375804040581e-05
Penalty params: tau=0.59838 conn_l=0.00100 val_l=0.00100 euler_l=1.00000 epoch=1115 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 1115, train
 fgw:0.2152536
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2152536
Measure Epoch 1115, train
 similarity:0.0282638
 penlog:-90.1507009
Metrics Epoch 1115, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:99.0434783
 batch_molecular_disconnected_validity:10.0000000
 batch_connected_components:3.8400000
 batch_invalid_valency_nodes:24.2608696
 batch_nodes_0degree:1.7800000
 batch_nodes_7plus_degree:0.5200000
 invalid_euler_toofew:34.0000000
 invalid_euler_toomany:6.0000000
 avg_euler_error:-0.2900000
 batch_node_degree:2.4713043
Logits [20.291175842285156, 2.3867151737213135, 68.3794174194336]
Epoch duration: 2.410834312438965
mol_opt/output_dev2/ffn-base-nopen-gumbel1/model_ffn-base-nopen-gumbel1_1115
Epoc

...

Epoch: 1484
FGW torch.Size([29508, 5]) 6.437198317144066e-05
Penalty params: tau=0.50486 conn_l=0.00100 val_l=0.00100 euler_l=1.00000 epoch=1484 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 1484, train
 fgw:0.1982079
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.1982079
Measure Epoch 1484, train
 similarity:0.0261687
 penlog:-94.1114585
Metrics Epoch 1484, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:99.0434783
 batch_molecular_disconnected_validity:6.0000000
 batch_connected_components:3.7000000
 batch_invalid_valency_nodes:22.2608696
 batch_nodes_0degree:1.4400000
 batch_nodes_7plus_degree:0.6200000
 invalid_euler_toofew:40.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-0.5300000
 batch_node_degree:2.5582609
Logits [19.200511932373047, 2.3941900730133057, 62.972557067871094]
Epoch duration: 2.3839943408966064
mol_opt/output_dev2/ffn-base-nopen-gumbel1/model_ffn-base-nopen-gum

Epoch: 1493
FGW torch.Size([29508, 5]) 6.51414884487167e-05
Penalty params: tau=0.50277 conn_l=0.00100 val_l=0.00100 euler_l=1.00000 epoch=1493 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 1493, train
 fgw:0.2008974
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2008974
Measure Epoch 1493, train
 similarity:0.0210526
 penlog:-94.0885367
Metrics Epoch 1493, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:99.1304348
 batch_molecular_disconnected_validity:6.0000000
 batch_connected_components:3.4800000
 batch_invalid_valency_nodes:23.9130435
 batch_nodes_0degree:1.6600000
 batch_nodes_7plus_degree:0.7600000
 invalid_euler_toofew:24.0000000
 invalid_euler_toomany:8.0000000
 avg_euler_error:0.6000000
 batch_node_degree:2.6295652
Logits [19.61166763305664, 2.3913183212280273, 63.02050018310547]
Epoch duration: 2.727903366088867
mol_opt/output_dev2/ffn-base-nopen-gumbel1/model_ffn-base-nopen-gumbel1_

Epoch: 1502
FGW torch.Size([29508, 5]) 6.584806396858767e-05
Penalty params: tau=0.50069 conn_l=0.00100 val_l=0.00100 euler_l=1.00000 epoch=1502 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 1502, train
 fgw:0.1955480
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.1955480
Measure Epoch 1502, train
 similarity:0.0379110
 penlog:-86.1232546
Metrics Epoch 1502, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:99.0434783
 batch_molecular_disconnected_validity:14.0000000
 batch_connected_components:3.8800000
 batch_invalid_valency_nodes:25.2173913
 batch_nodes_0degree:1.6000000
 batch_nodes_7plus_degree:0.9000000
 invalid_euler_toofew:28.0000000
 invalid_euler_toomany:2.0000000
 avg_euler_error:0.2800000
 batch_node_degree:2.6643478
Logits [20.009492874145508, 2.5429699420928955, 65.05558776855469]
Epoch duration: 2.3699824810028076
mol_opt/output_dev2/ffn-base-nopen-gumbel1/model_ffn-base-nopen-gumb

Epoch: 1511
FGW torch.Size([29508, 5]) 6.656096229562536e-05
Penalty params: tau=0.49862 conn_l=0.00100 val_l=0.00100 euler_l=1.00000 epoch=1511 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 1511, train
 fgw:0.1972249
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.1972249
Measure Epoch 1511, train
 similarity:0.0207692
 penlog:-96.0507562
Metrics Epoch 1511, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:99.0434783
 batch_molecular_disconnected_validity:4.0000000
 batch_connected_components:2.6000000
 batch_invalid_valency_nodes:26.0000000
 batch_nodes_0degree:1.3400000
 batch_nodes_7plus_degree:0.9600000
 invalid_euler_toofew:12.0000000
 invalid_euler_toomany:20.0000000
 avg_euler_error:2.3000000
 batch_node_degree:2.8226087
Logits [19.626070022583008, 2.4608750343322754, 64.98458099365234]
Epoch duration: 2.5520453453063965
mol_opt/output_dev2/ffn-base-nopen-gumbel1/model_ffn-base-nopen-gumb

RuntimeError: [enforce fail at inline_container.cc:262] . unexpected pos 194218432 vs 194218320

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

In [29]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

tensor(27.7234, device='cuda:0', grad_fn=<AddBackward0>)

In [30]:
torch.optim.Adam([torch.autograd.Variable(torch.Tensor([0.]))]).param_groups[0]['lr']

0.001

In [2]:
99 ** (1/10)

1.5833011217497763

In [28]:
vars(args)

{'cuda': True,
 'output_dir': 'mol_opt/output_dev1/pointwise-test5',
 'tb_logs_dir': 'mol_opt/logs_dev1/pointwise-test5',
 'init_model': 'pointwise-test5',
 'init_decoder_model': 'pointwise-test5_decode',
 'task': 'qed',
 'model_type': 'pointwise',
 'one_batch_train': True,
 'batch_size': 50,
 'pred_hidden': 150,
 'pc_hidden': 100,
 'ffn_activation': 'LeakyReLU',
 'n_epochs': 1000,
 'dim_tangent_space': 40,
 'n_layers': 5,
 'n_hidden': 250,
 'n_ffn_hidden': 100,
 'linear_out': False,
 'dropout_gcn': 0.0,
 'dropout_ffn': 0.0,
 'agg_func': 'sum',
 'batch_norm': False,
 'N_transformer': 6,
 'n_ffn_transformer': 100,
 'n_heads_transformer': 10,
 'dropout_transformer': 0.1,
 'ot_solver': 'emd',
 'sinkhorn_entropy': 0.1,
 'sinkhorn_max_it': 10000,
 'connectivity': False,
 'valency': True,
 'euler_characteristic_penalty': True,
 'annealing_rate': 0.002,
 'connectivity_lambda': 0,
 'valency_lambda': 0,
 'euler_lambda': 1,
 'connectivity_hard': False,
 'valency_hard': False,
 'scale_lambdas': F

In [32]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("/home/octav/gitrepos/tum-thesis/mol_opt/dev7")

In [31]:
tags = [
    "pointwise10-dev8/train_avg_euler_error/mean",
    "pointwise10-dev8/train_avg_euler_error/bot_band",
    "pointwise10-dev8/train_avg_euler_error/top_band"
]
# writer.add_custom_scalars_marginchart(tags)


In [2]:
# writer.add_scalar("plm2/plm2", 0, 1)
# writer.add_scalar("plm2/plm2", 1, 2)
# writer.add_scalar("plm2/plm2", 0, 3)
# writer.add_scalar("plm2/plm2", 1, 4)
tags = ["plm", "plm", "plm"]
# layout = {"plm" : {"plm" : ["Margin", tags]}}
layout = {"plm1" : {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]},
          "plm2": {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]}}
writer.add_custom_scalars(layout)

In [33]:
writer._get_file_writer().add_summary()

TypeError: add_summary() missing 1 required positional argument: 'summary'

In [34]:
writer.logdir

'/home/octav/gitrepos/tum-thesis/mol_opt/dev6'

In [11]:
writer.close()

In [4]:
import tensorboardX
tensorboardX.__version__

'2.1'

In [6]:
from tensorboardX.summary import custom_scalars

In [16]:
writer._get_file_writer().add_summary(custom_scalars(layout), global_step = 3)
writer.flush()

In [34]:
writer.add_scalars('plm', {'plm0' : 0.0, 'plmt' : 0.1, 'plmb' : -0.1}, 3)
writer.flush()

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [ ]:
molopt.state_dict()

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)